# Results of ResNet18 Trained on CIFAR100

In this notebook I display results in terms of accuracy on test data for torchvisions' ResNet18 models trained on the CIFAR100 dataset using various regularization techniques. The CIFAR100 data is preprocessed by normalizing using mean [0.5071, 0.4865, 0.4409] and variance [0.2009, 0.1984, 0.2023]. The batch size is 100. The model optimizes using SGD with momentum p = 0.9, and standard cross-entropy loss. Models are trained with no regularization, L2 regularization, SVB regularization and Jacobian regularization (there is no dropout for the standard ResNet models). The weights area initialized by the pytorch standard. The L2 regularization coefficient and Jacobian regularization coefficient are the same as in Hoffman 2019: l2_lmbd = 0.0005 and lambda_jacobian_reg = 0.01. For SVB regularization I use the hyperparameters svb_freq=600 and svb_eps = 0.05. The learning rate starts at 0.1, and is reduced to 0.01 and 0.001 1/3 and 2/3s into training, respectively. The models are trained for 50 epochs.

### Imports and Model Loading

In [1]:
import jupyter_black
import numpy as np
import pandas as pd
import pickle
import torch
from scipy import stats
from torchsummary import summary
from tqdm import tqdm

from data_generators import data_loader_CIFAR100
from model_classes import ResNet18
from tools import accuracy, ModelInfo

jupyter_black.load()

In [2]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load MNIST data
train_loader, test_loader = data_loader_CIFAR100()

# Summary of model
summary_model = ResNet18().to(device)
summary(summary_model, (3, 32, 32))

Files already downloaded and verified
Files already downloaded and verified
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 16, 16]           9,408
       BatchNorm2d-2           [-1, 64, 16, 16]             128
              ReLU-3           [-1, 64, 16, 16]               0
         MaxPool2d-4             [-1, 64, 8, 8]               0
            Conv2d-5             [-1, 64, 8, 8]          36,864
       BatchNorm2d-6             [-1, 64, 8, 8]             128
              ReLU-7             [-1, 64, 8, 8]               0
            Conv2d-8             [-1, 64, 8, 8]          36,864
       BatchNorm2d-9             [-1, 64, 8, 8]             128
             ReLU-10             [-1, 64, 8, 8]               0
       BasicBlock-11             [-1, 64, 8, 8]               0
           Conv2d-12             [-1, 64, 8, 8]          36,864
      BatchNorm2d-13       

c:\ProgramData\Anaconda3\envs\bioai\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\ProgramData\Anaconda3\envs\bioai\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [3]:
# Load models
dataset = "cifar100"

model_names_set = [
    "model_no_reg",
    "model_l2",
    "model_jacobi",
    "model_svb",
]
model_names = []

for i in range(5):
    for name in model_names_set:
        model_names.append(f"{name}_{i}")

models = {name: ModelInfo(name, dataset) for name in model_names}

### Accuracies

Here I calculate the accuracies of each model as a 95% confidence interval. I have trained five models of each variation that I use to calculate the CI.

In [4]:
# Dictionary to hold accuracy data
accuracy_data = {}

# Calculate accuracies with 95% CI for models
for name in model_names_set:
    accuracies = []
    for i in range(5):  # Assuming 5 versions of each model
        model_name = f"{name}_{i}"
        model = models[model_name].model
        acc = accuracy(model, test_loader, device)
        accuracies.append(acc)

    # Calculate mean accuracy
    mean_accuracy = np.mean(accuracies)

    # Calculate standard error
    std_error = stats.sem(accuracies)

    # Calculate confidence interval
    CI = std_error * stats.t.ppf((1 + 0.95) / 2, len(accuracies) - 1)

    # Store results in the data dictionary
    accuracy_data[f"{name}"] = (mean_accuracy, CI)

In [5]:
# Convert the dictionary into a DataFrame
df = pd.DataFrame.from_dict(
    accuracy_data, orient="index", columns=["Mean Accuracy", "Confidence Interval"]
)

# Convert the numbers from decimal to percentage and round to four decimal places
df = df.multiply(100).round(3)

# Create a new column that combines Mean Accuracy and Confidence Interval
df["Accuracy +/- CI (95%)"] = df.apply(
    lambda row: f"{row['Mean Accuracy']} +/- {row['Confidence Interval']}", axis=1
)

# Drop the original columns
df = df.drop(columns=["Mean Accuracy", "Confidence Interval"])

# Reset the index
df = df.reset_index().rename(columns={"index": "Model"})

display(df)

,Model,Accuracy +/- CI (95%)
0,model_no_reg,45.338 +/- 0.54
1,model_l2,47.572 +/- 0.982
2,model_jacobi,44.478 +/- 0.52
3,model_svb,48.278 +/- 0.615


### Total Variation

Total variation is a measure for roughness/complexity in images. I generate 25 different images for each model to get a good mean and standard deviation, and give the results as a tabel for each model with the eight models (models with and without dropout) as rows, and the three different zoom levels as columns. The tabel contains the mean and 95 % confidence interval for the total variation for each model at each zoom level, for both isotropic and anisotropic total variation.

#### Isotropic Total Variation

In [6]:
# Load pretrained results

# Define zoom levels
zoom_levels = [0.025, 0.01, 0.001]

# Dataframe to store results
cols = pd.MultiIndex.from_product([zoom_levels, ["mean", "conf_interval"]])
df_results_isotropic = pd.DataFrame(index=model_names, columns=cols)

# Loop over the selected models
for name in tqdm(model_names):
    # Load the results
    with open(
        f"./total_variation_{dataset}_models/{name}_total_isotropic_variation.pkl",
        "rb",
    ) as f:
        df_results = pickle.load(f)

    # Copy the results to the dataframe
    for zoom in zoom_levels:
        df_results_isotropic.loc[name, (zoom, "mean")] = df_results.loc[
            name, (zoom, "mean")
        ]
        df_results_isotropic.loc[name, (zoom, "conf_interval")] = df_results.loc[
            name, (zoom, "conf_interval")
        ]

  0%|          | 0/20 [00:00<?, ?it/s]

100%|██████████| 20/20 [00:00<00:00, 91.11it/s]


In [7]:
print(
    "Isotropic Total variation for three different zoom levels (mean and 95% CI for 50 generated images at each zoom level)"
)
display(df_results_isotropic)

Isotropic Total variation for three different zoom levels (mean and 95% CI for 50 generated images at each zoom level)


0.025                       0.010                \
                        mean conf_interval          mean conf_interval   
model_no_reg_0  37963.594568   7501.297261  61802.605519   7840.443753   
model_l2_0      36847.282305   6956.196202    67696.1996   6095.020443   
model_jacobi_0  33530.704302   6688.738272  62654.231511   7193.654255   
model_svb_0     32118.541484   7720.606059  60791.477421   6940.035441   
model_no_reg_1  36727.288706   6376.536743  63664.253752   7818.495468   
model_l2_1      38009.502152   7650.767029  61120.742575   7210.230465   
model_jacobi_1  34826.634875   7358.276433  63518.565686   9123.684654   
model_svb_1     33223.002129   7133.782178  55521.797616   6922.027032   
model_no_reg_2  28911.768857   6872.254659  58904.662862   8712.701742   
model_l2_2      39003.440135   6566.281239  68893.401165   7146.482009   
model_jacobi_2  37071.364369   6395.065478  64588.013044   7273.449077   
model_svb_2     39155.101637   7685.720982  60355.604887   8197.503567   
model_no_reg_3   31455.54922   6046.818199  62082.369167   7486.789545   
model_l2_3      32514.949247   8255.610707  60268.500837    7879.89621   
model_jacobi_3  32250.010467   7173.283096  53878.139702   7274.206496   
model_svb_3     36321.155917   7176.044977  62426.346296   8366.155183   
model_no_reg_4  34327.422938   6965.397246  61932.118761   8370.654704   
model_l2_4      40768.745438   7093.366383  67816.696541   8290.859095   
model_jacobi_4  39494.629802   8951.808079  56299.056856   7454.183924   
model_svb_4     35105.212475   6991.869529  57460.472128   7614.327547   

                       0.001                
                        mean conf_interval  
model_no_reg_0   15111.28083   1917.137481  
model_l2_0      30004.745481   2474.079136  
model_jacobi_0  29676.655099   2245.506765  
model_svb_0     25374.445924   3944.470671  
model_no_reg_1  12449.370343   1628.248709  
model_l2_1      21648.677742   1857.702156  
model_jacobi_1   15383.66227   1555.031038  
model_svb_1      9066.354813   1210.719408  
model_no_reg_2  27152.547901   2322.470937  
model_l2_2      48255.025899   5413.974684  
model_jacobi_2  54416.506484    3787.23314  
model_svb_2     66904.388381    4747.47271  
model_no_reg_3  19689.441415   1727.265354  
model_l2_3      52637.610688   5191.357856  
model_jacobi_3  43879.173863   4397.617203  
model_svb_3     30982.197412   3225.922996  
model_no_reg_4  35982.847688   3568.021995  
model_l2_4      15583.415754   2682.432029  
model_jacobi_4   19603.18449   2672.604959  
model_svb_4     20122.935573   2542.639682

#### Anisotropic Total Variation

In [8]:
# Load pretrained results

# Define zoom levels
zoom_levels = [0.025, 0.01, 0.001]

# Dataframe to store results
cols = pd.MultiIndex.from_product([zoom_levels, ["mean", "conf_interval"]])
df_results_anisotropic = pd.DataFrame(index=model_names, columns=cols)

# Loop over the selected models
for name in tqdm(model_names):
    # Load the results
    with open(
        f"./total_variation_{dataset}_models/{name}_total_anisotropic_variation.pkl",
        "rb",
    ) as f:
        df_results = pickle.load(f)

    # Copy the results to the dataframe
    for zoom in zoom_levels:
        df_results_anisotropic.loc[name, (zoom, "mean")] = df_results.loc[
            name, (zoom, "mean")
        ]
        df_results_anisotropic.loc[name, (zoom, "conf_interval")] = df_results.loc[
            name, (zoom, "conf_interval")
        ]

  0%|          | 0/20 [00:00<?, ?it/s]

100%|██████████| 20/20 [00:00<00:00, 102.07it/s]


In [9]:
print(
    "Anisotropic Total variation for three different zoom levels (mean and 95% CI for 50 generated images at each zoom level)"
)
display(df_results_anisotropic)

Anisotropic Total variation for three different zoom levels (mean and 95% CI for 50 generated images at each zoom level)


0.025                   0.010                   0.001  \
                    mean conf_interval      mean conf_interval      mean   
model_no_reg_0   41718.8     8221.0674  67872.76   8638.503822  16719.12   
model_l2_0      40358.86    7602.91549  74259.56   6641.454819  33054.04   
model_jacobi_0   37001.3   7336.831948  68799.12   7947.408602   32719.3   
model_svb_0      35168.4   8451.821394   66577.0   7648.599211  27943.24   
model_no_reg_1  40333.04   6985.907618  70040.38   8596.028789  13832.08   
model_l2_1      41916.78   8428.635204  67202.88   7914.416121  23892.12   
model_jacobi_1  38590.82   8215.009295  69848.42   9983.438765  16962.98   
model_svb_1      36261.3   7797.267222  60798.64   7583.824248  10062.16   
model_no_reg_2   31676.7   7450.115787  64388.62   9445.972917  29976.56   
model_l2_2      42444.08   7163.380032  75620.68   7879.418261  53253.66   
model_jacobi_2  40429.76   6872.760426  70877.12   7963.703318  59849.72   
model_svb_2     43046.12   8388.659742  66426.78   9012.966437  73701.88   
model_no_reg_3   34521.1   6719.118428  68375.36   8286.179892   21872.4   
model_l2_3      35715.44   9050.732822  66221.96   8666.860543   58190.4   
model_jacobi_3  35416.72   7848.494689  59020.04   7897.654354  48337.52   
model_svb_3      39968.2    7884.62388   68452.0   9235.281028  34176.02   
model_no_reg_4   37628.6     7646.9308  67883.72   9142.538287  39642.38   
model_l2_4      45107.96   7864.296375  74650.62   9147.349838  17236.68   
model_jacobi_4  43323.44   9788.278394  61784.26    8200.90015   21749.7   
model_svb_4      38481.7   7651.014377  62897.72   8297.985624   22099.0   

                              
               conf_interval  
model_no_reg_0   2101.589632  
model_l2_0       2713.389792  
model_jacobi_0   2441.058804  
model_svb_0      4319.854694  
model_no_reg_1    1786.99555  
model_l2_1       2031.084606  
model_jacobi_1   1700.853737  
model_svb_1       1349.18417  
model_no_reg_2   2566.246137  
model_l2_2       5881.886287  
model_jacobi_2   4074.739739  
model_svb_2      5119.964615  
model_no_reg_3   1968.278474  
model_l2_3       5808.226853  
model_jacobi_3   4920.675735  
model_svb_3      3590.899831  
model_no_reg_4   3982.281953  
model_l2_4       2929.869512  
model_jacobi_4    2936.92328  
model_svb_4      2745.720829